# recombination.py
This module contains two functions which are used to create and mutate children as part of a Genetic Algorithm (GA).
The first function is called create_children, and it in turn calls the second function, mutate.  Generally an external module need only call create_children directly.

In [1]:
import numpy as np
from numpy.random import seed
from numpy.random import randint
from sys import exit
import constraints
import copy

Creates children timetable solutions from parent generation. Checks a global to see what type of crossover scheme should be used. Returns a list of children solutions, created from parents.  Each child is represented by a dictionary with keys that are course names, and values that are another dictionary.  Within the nested dictionary, keys are time, room, and prof.  For example, a single solution may look like this:
{'CISC101': {'time': 1, 'room': 1, 'prof': 1}, 'CISC102': {'time': 0, 'room': 4, 'prof': 1}, 'CISC103': {'time': 3, 'room': 0, 'prof': 2}}
        
And a list of three children could therefore look like:

        [{'CISC101': {'time': 1, 'room': 1, 'prof': 1}, 'CISC102': {'time': 0, 'room': 4, 'prof': 1}, 'CISC103': {'time': 3, 'room': 0, 'prof': 3}}, 
        {'CISC101': {'time': 0, 'room': 2, 'prof': 1}, 'CISC102': {'time': 0, 'room': 1, 'prof': 2}, 'CISC103': {'time': 0, 'room': 0, 'prof': 2}}, 
        {'CISC101': {'time': 2, 'room': 1, 'prof': 3}, 'CISC102': {'time': 0, 'room': 2, 'prof': 1}, 'CISC103': {'time': 3, 'room': 2, 'prof': 2}}]

If there is a "Fitness" key within the parent, it is returned as well.  For example a single child could also look like this:
{'CISC101': {'time': 1, 'room': 1, 'prof': 1}, 'CISC102': {'time': 0, 'room': 4, 'prof': 1}, 'CISC103': {'time': 3, 'room': 0, 'prof': 2}, 'Fitness': 95}     

In [2]:
def create_children(num_rooms, num_times, parents, fitnesses, num_children, populationcopy):
    children = []
    for j in range(num_children):
        if constraints.recombtype == "clone":
            # clone simply selects a parent at random, and returns a copy of that parent as a child
            parent_key = randint(0,len(parents))
            newpop = populationcopy[:]
            newchild = newpop[parents[parent_key]].copy()

            # now mutate the child to introduce random variance in the subsequent population
            mutechild = mutate(newchild, num_rooms, num_times, constraints.mutate_chance)     
            children.append(mutechild)
    return children

Mutates a given timetable solutions, to introduce variance within a Genetic Algorithm. Applies a mutation scheme. Returns a single children solutions, created as a result of mutation calculations.  Each child is represented by a dictionary with keys that are course names, and values that are another dictionary.  Within the nested dictionary, keys are time, room, and prof. For example, a single solution may look like this:
                 
      {'CISC101': {'time': 1, 'room': 1, 'prof': 1}, 'CISC102': {'time': 0, 'room': 4, 'prof': 1}, 'CISC103': {'time': 3, 'room': 0, 'prof': 2}}

If there is a "Fitness" key within the solution provided, it is not mutated and returned as well.  For example a returned solution could also look like this:

    {'CISC101': {'time': 1, 'room': 1, 'prof': 1}, 'CISC102': {'time': 0, 'room': 4, 'prof': 1}, 'CISC103': {'time': 3, 'room': 0, 'prof': 2}, 'Fitness': 95}  

In [3]:
def mutate(child_param, num_rooms, num_times, mutatechance):
    child = copy.deepcopy(child_param)
    for course in child:
        # This makes sure we ignore the Fitness key within a solution, as it is calculated from other values
        if course == 'Fitness':
            exit
        else:
            # Apply room mutation
            if np.random.rand() < mutatechance:
                # 50% chance of mutation going up or down.  If max/min value is already reached, do nothing
                if np.random.rand() < .5:
                    if child[course]['room'] >= num_rooms-1:
                        pass
                    else:
                        child[course]['room'] += 1
                else:
                    if child[course]['room'] < 1:
                        pass
                    else:
                        child[course]['room'] -= 1
            else:
                pass
            # Apply time mutation
            if np.random.rand() < mutatechance:
                # 50% chance of mutation going up or down.  If max/min value is already reached, do nothing
                if np.random.rand() < .5:
                    if child[course]['time'] >= num_times-1:
                        pass
                    else:
                        child[course]['time'] += 1
                else:
                    if child[course]['time'] < 1:
                        pass
                    else:
                        child[course]['time'] -= 1
            else:
                pass
    return child
